In [1]:
import cv2
import numpy as np 
import pandas as pd 
import os
from PIL import Image
from tensorflow.keras.applications.resnet50 import ResNet50
import tensorflow as tf
import tensorflow.keras as K
from tensorflow.keras import datasets, layers, models, losses, Model
from sklearn.model_selection import train_test_split
from sklearn import preprocessing as pre
from glob import glob
from sklearn.utils.class_weight import compute_class_weight
os.environ["CUDA_VISIBLE_DEVICES"] = '4'


2023-07-27 19:08:38.801074: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-07-27 19:08:39.554332: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
Train_img_list=pd.read_csv('../../data/standardFrame_data/scale_skip/Train_dataframe.csv')['file_path'].to_list()
Train_label_list=pd.read_csv('../../data/standardFrame_data/scale_skip/Train_dataframe.csv')['standard'].to_list()
Test_img_list=pd.read_csv('../../data/standardFrame_data/scale_skip/Test_dataframe.csv')['file_path'].to_list()
Test_label_list=pd.read_csv('../../data/standardFrame_data/scale_skip/Test_dataframe.csv')['standard'].to_list()
Val_img_list=pd.read_csv('../../data/standardFrame_data/scale_skip/Validation_dataframe.csv')['file_path'].to_list()
Val_label_list=pd.read_csv('../../data/standardFrame_data/scale_skip/Validation_dataframe.csv')['standard'].to_list()

In [3]:
Train_img_path='../../data/standardFrame_data/scale_skip/train'
Test_img_path='../../data/standardFrame_data/scale_skip/test'
Val_img_path='../../data/standardFrame_data/scale_skip/val'

In [4]:
size=224
x_train = np.zeros((len(Train_img_list),size,size,3))
for i in range(len(Train_img_list)):
    x_train[i] =np.array(Image.open(Train_img_path+Train_img_list[i]).resize((size,size)))
x_train=x_train/255
y_train=np.array(Train_label_list)

x_test = np.zeros((len(Test_img_list),size,size,3))
for i in range(len(Test_img_list)):
    x_test[i] =np.array(Image.open(Test_img_path+Test_img_list[i]).resize((size,size)))
x_test=x_test/255
y_test=np.array(Test_label_list)

x_val = np.zeros((len(Val_img_list),size,size,3))
for i in range(len(Val_img_list)):
    x_val[i] =np.array(Image.open(Val_img_path+Val_img_list[i]).resize((size,size)))
x_val=x_val/255
y_val=np.array(Val_label_list)

In [5]:

checkpoint_filepath = "../../model/skip/MobileNetV2_checkpoints.h5"
model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_filepath,
    save_best_only= True
)
class_weight_ratio=compute_class_weight(class_weight = "balanced" , 
                     classes=np.unique(y_train), 
                     y = y_train)
class_weight = {0:class_weight_ratio[0],1:class_weight_ratio[1]}

input_t=K.Input(shape=(size,size, 3))
input_tensor = layers.experimental.preprocessing.Resizing(size, size, interpolation="bilinear", input_shape=x_train.shape[1:])(input_t)
ResNet=K.applications.MobileNetV2(include_top=True,weights='imagenet',input_tensor=input_tensor)
model = K.models.Sequential()
model.add(ResNet)
model.add(tf.keras.layers.Dropout(.2, input_shape=(64,)))
model.add(K.layers.Dense(64, activation=tf.keras.layers.LeakyReLU(alpha=0.1)))
model.add(K.layers.Dense(1, activation='sigmoid'))
model.compile(optimizer=K.optimizers.Adam(lr=2e-3),
                loss=tf.keras.losses.binary_crossentropy,
                metrics=["accuracy"])
histo=model.fit(
    x_train,y_train,
    validation_data=(x_val,y_val),
    epochs=500,
    callbacks=[model_checkpoint_callback],
    batch_size=32,shuffle=True,
    class_weight=class_weight
)
model.save('../../model/skip/MobileNetV2.h5')

2023-07-27 19:11:22.373097: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1635] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 38163 MB memory:  -> device: 0, name: NVIDIA A100-PCIE-40GB, pci bus id: 0000:81:00.0, compute capability: 8.0


14536120/14536120 [==============================] - 2s 0us/step


2023-07-27 19:11:29.542697: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 14973923328 exceeds 10% of free system memory.
2023-07-27 19:11:36.434101: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 14973923328 exceeds 10% of free system memory.


Epoch 1/500


2023-07-27 19:11:49.561001: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:424] Loaded cuDNN version 8902
2023-07-27 19:11:50.473151: I tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:637] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.
2023-07-27 19:11:50.476299: I tensorflow/compiler/xla/service/service.cc:169] XLA service 0x7f04087aac70 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2023-07-27 19:11:50.476324: I tensorflow/compiler/xla/service/service.cc:177]   StreamExecutor device (0): NVIDIA A100-PCIE-40GB, Compute Capability 8.0
2023-07-27 19:11:50.479738: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:269] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2023-07-27 19:11:50.592208: I ./tensorflow/compiler/jit/device_compiler.h:180] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the pro

778/778 [==============================] - 65s 52ms/step - loss: 0.6357 - accuracy: 0.5757 - val_loss: 0.3358 - val_accuracy: 0.9335
Epoch 2/500
778/778 [==============================] - 32s 41ms/step - loss: 0.4742 - accuracy: 0.7472 - val_loss: 1.6252 - val_accuracy: 0.0665
Epoch 3/500
778/778 [==============================] - 32s 41ms/step - loss: 0.3509 - accuracy: 0.7978 - val_loss: 1.5685 - val_accuracy: 0.2422
Epoch 4/500
778/778 [==============================] - 32s 41ms/step - loss: 0.3285 - accuracy: 0.8281 - val_loss: 2.2049 - val_accuracy: 0.0917
Epoch 5/500
778/778 [==============================] - 32s 41ms/step - loss: 0.2682 - accuracy: 0.8526 - val_loss: 1.2601 - val_accuracy: 0.5151
Epoch 6/500
778/778 [==============================] - 32s 41ms/step - loss: 0.2444 - accuracy: 0.8810 - val_loss: 1.1874 - val_accuracy: 0.5205
Epoch 7/500
778/778 [==============================] - 32s 41ms/step - loss: 0.2120 - accuracy: 0.8957 - val_loss: 2.3142 - val_accuracy: 0.19